In [1]:

import random
import networkx as nx
import matplotlib.pyplot as plt


In [3]:
def createGridGraph(sideLength):
    maxX = sideLength
    maxY = maxX

    network = nx.grid_2d_graph(maxX,maxY)

    newLocations ={}
    for node in network.nodes(): 
        newLocation = (node[0]/maxY*100,node[1]/maxX*100)
        newLocations[node] = newLocation

    nx.set_node_attributes(network,'pos',newLocations)
    pos = nx.get_node_attributes(network,'pos')
    
    nx.set_edge_attributes(network,'weight',100)
    nx.set_edge_attributes(network,'color','black')
    return network


In [4]:
def getRandomNode(graph):
    node = random.choice(graph.nodes())
    return node

In [5]:
def getRandomSourceTarget(graph,source,target):
    if source == None and target == None:
        rand1 = getRandomNode(graph)
        rand2 = getRandomNode(graph)
    elif source  == None or target == None:
        if source == None:
            rand1 = source
            rand2= getRandomNode(graph)
        else:
            rand1 = target
            rand2 = getRandomNode(graph)        
        
        
    while rand1 == rand2:
        rand2 = getRandomNode(graph)
        
    sourceTarget = (rand1,rand2)
    return sourceTarget

In [6]:
def getPath(graph,source=None,target=None):
    (source,target) = getRandomSourceTarget(graph,source,target)
    path = nx.shortest_path(graph,source=source,target=target,weight='weight')
    path_edges = list(zip(path[:-1],path[1:]))
    return path_edges

In [7]:
def adjustWeights(graph,path,weightAdjust=1,minWeight=None):
    for v in path:
        if not graph.has_edge(*v):
            v=(v[1],v[0])
        graph.get_edge_data(*v)['weight'] = graph.get_edge_data(*v)['weight']*weightAdjust
        if minWeight is not None and graph.get_edge_data(*v)['weight'] < minWeight:
            graph.get_edge_data(*v)['weight'] = minWeight

    

In [8]:
def displayGraphPath(network,path=None):
    
    nx.set_edge_attributes(network,'color','black')
    edge_colors = nx.get_edge_attributes(network,'weight')
    
    pos = nx.get_node_attributes(network,'pos')
    edgeColorList = list(edge_colors.values())
    edgeList = list(edge_colors.keys())
    nx.draw_networkx_edges(network,pos,width=5,edge_color=edgeColorList,edgelist=edgeList,edge_cmap=plt.cm.Blues_r)
    
    
#    if path is not None:
#        nx.draw_networkx_edges(network,pos,width=10,edge_color='red',edgelist=path)
#        path_edges_colors = dict.fromkeys(path,'red')
    
#print(edge_colors.keys())
#print(edge_colors.values())
    plt.show()

In [9]:
def correctWeightError(network,defaultWeight=100,errorCorrectThreshold=2,minWeight=50,maxWeight=200):
    weightAttributes = nx.get_edge_attributes(network,'weight')
    
    weightList = list(weightAttributes.values())
    
    #only need to calculate once
    numOfEdges = len(weightList)
    totalCorrectWeight = numOfEdges*defaultWeight
    
    weightList = list(weightAttributes.values())
    sumOfWeights = sum(weightList)
    averageError = (sumOfWeights-totalCorrectWeight)/numOfEdges
    print(averageError)
    
    while abs(averageError) > errorCorrectThreshold:
        print("performing error correct")
        for k,v in weightAttributes.items():
            weightAttributes[k] -= averageError
            if weightAttributes[k] < minWeight:
                weightAttributes[k] = minWeight
            if weightAttributes[k] > maxWeight:
                weightAttributes[k] = maxWeight
            
            
        weightList = list(weightAttributes.values())
        sumOfWeights = sum(weightList)
        averageError = (sumOfWeights-totalCorrectWeight)/numOfEdges
    
    nx.set_edge_attributes(network,'weight',weightAttributes)
    
    

In [10]:
def randomizeWeights(network,minWeight=1,maxWeight=400):
    #apply random weights
    weights = nx.get_edge_attributes(network,'weight')
    for w in weights.keys():
        weights[w] = random.uniform(minWeight,maxWeight)
    nx.set_edge_attributes(network,'weight',weights)


In [11]:
def saveImage(graph,filename):
    edge_colors = nx.get_edge_attributes(graph,'weight')
    pos = nx.get_node_attributes(graph,'pos')
    edgeColorList = list(edge_colors.values())
    edgeList = list(edge_colors.keys())
   
    nx.draw_networkx_edges(graph,pos,width=5,edge_color=edgeColorList,edgelist=edgeList,edge_cmap=plt.cm.Blues_r)
    plt.savefig(filename)

In [12]:

def runProgram():
    minWeight = 5

    G = createGridGraph(25)
    randomizeWeights(G,minWeight=95,maxWeight=105)
    correctWeightError(G)

    for a in range(20):
        print(a)
        for x in range(100):
        #    print(str(a) + "-" + str(x))
            path = getPath(G)
            
            adjustWeights(G,path=path,minWeight=minWeight,weightAdjust=.7)
            
        correctWeightError(G)
        base = r"C:\Users\Joe\Documents\Joe\Google Drive\Coding\ipython\Highways\testing_folder\joe_"
        filename = base + str(a) + ".png"
        saveImage(G,filename)

In [13]:
import cProfile
cProfile.run("runProgram()")
#runProgram()

0.0963504304116456
0
-14.783100494494077
performing error correct
performing error correct
1
-10.970389952770239
performing error correct
performing error correct
2
-9.903548042927117
performing error correct
performing error correct
3
-9.487661231334968
performing error correct
performing error correct
4
-8.681884200654617
performing error correct
performing error correct
5
-9.367415576782538
performing error correct
performing error correct
6
-8.991175585647166
performing error correct
performing error correct
7
-7.808387568780588
performing error correct
performing error correct
8
-7.761822581374302
performing error correct
performing error correct
9
-8.051423733270518
performing error correct
performing error correct
10
-7.898296727134281
performing error correct
performing error correct
11
-7.904062150692153
performing error correct
performing error correct
12
-7.536062530170033
performing error correct
performing error correct
13
-7.579587098760846
performing error correct
perfor

C:\Users\Joe\Anaconda3\lib\site-packages\matplotlib\collections.py:650: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors_original != str('face'):
C:\Users\Joe\Anaconda3\lib\site-packages\matplotlib\collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [25]:


#network = createGridGraph(30)

nx.get_edge_attributes(network,'weight')

NameError: name 'network' is not defined

In [ ]:
minWeight = 5

network = createGridGraph(2)
correctWeightError(network)

for a in range(4):
    for x in range(3):
        print(str(a) + "-" + str(x))
        path = getPath(network)
        adjustWeights(network,path=path,minWeight=minWeight,weightAdjust=.7)
        #correctWeightError(network)

        #displayGraphPath(network,path)
    base = r"C:\Users\Joe\Documents\Joe\Google Drive\Coding\ipython\Highways\testing_folder\joe_"
    filename = base + str(a) + ".png"
  #  print(nx.get_edge_attributes(network,'weight'))
    saveImage(network,filename)

In [ ]:


network = createGridGraph(3)


path = getPath(network)
print(path)

print(network.edges(data=True))
displayGraphPath(network,path,highlightPath=True)

displayGraphPath(network,path,highlightPath=False)
#adjustWeights(network,path)

for v in path:
    print(v)
    if not network.has_edge(*v):
           v=(v[1],v[0])
    network.get_edge_data(*v)['weight'] = 50 #network.get_edge_data(*v)['weight']*weightAdjust

    
    

displayGraphPath(network,path,highlightPath=False)

print(network.edges(data=True))